In [ ]:
  !pip install PyPDF2 spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00


In [ ]:
# USING ACCESS TOKENS FROM HUGGING FACE AND LEGAL PEGASUS
!pip install transformers PyPDF2

In [ ]:
import os
import PyPDF2
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import papermill as pm

# Step 1: Extract text from the PDF
def extract_text_from_pdf(pdf_file_path):
    with open(pdf_file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in range(len(reader.pages)):
            text += reader.pages[page].extract_text()
    return text

# Step 2: Load the Legal Pegasus Model and Tokenizer
def load_pegasus_model():
    model_name = "Your_legalpegasusmodel or any other model"  # Replace with your specific model
    token = "your hugging face token or any other method preferred"  # Replace with your Hugging Face token
    tokenizer = PegasusTokenizer.from_pretrained(model_name, use_auth_token=token)
    model = PegasusForConditionalGeneration.from_pretrained(model_name, use_auth_token=token)
    return tokenizer, model

# Step 3: Summarize the text
def summarize_text(text, tokenizer, model, max_length=400):
    inputs = tokenizer(text, truncation=True, padding="longest", return_tensors="pt")
    summary_ids = model.generate(
        inputs.input_ids,
        max_length=512,
        num_beams=5,
        length_penalty=2.0,  # Penalize longer outputs
        repetition_penalty=2.5,  # Encourage diverse output
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Step 4: Remove repetitions from the summary
def remove_repetition(summary):
    lines = summary.split('. ')
    seen = set()
    filtered_summary = []
    for line in lines:
        if line not in seen:
            filtered_summary.append(line)
            seen.add(line)
    return '. '.join(filtered_summary)

# Step 5: Execute the notebook with Papermill
def execute_notebook_with_pdf(pdf_file_path):
    # Assuming you have a notebook ready with a code structure similar to what you provided
    input_notebook = 'HackOdisha_prototype1.ipynb'
    output_notebook = 'output_notebook.ipynb'

    # Run the notebook and pass the PDF input path as a parameter
    pm.execute_notebook(
        input_notebook,
        output_notebook,
        parameters=dict(pdf_input=pdf_file_path)  # Passing the path to the PDF
    )

    # Extract the summary from the output notebook (assuming it writes to a file or return summary)
    with open('output.txt', 'r') as summary_file:
        summary = summary_file.read()
    return summary

# Step 6: Integrate everything
def generate_legal_summary(pdf_file_path):
    # Extract the text from the PDF
    document_text = extract_text_from_pdf(pdf_file_path)

    # Execute the notebook (using papermill) with the PDF as input
    execute_notebook_with_pdf(pdf_file_path)

    # Alternatively, if you are using the Pegasus model locally:
    tokenizer, model = load_pegasus_model()
    summary = summarize_text(document_text, tokenizer, model)

    # Remove repetitions from the generated summary
    filtered_summary = remove_repetition(summary)

    return filtered_summary


# Example: Handling a file upload and summarization in a Flask app
from flask import Flask, request, render_template, redirect, url_for
from werkzeug.utils import secure_filename

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = './uploads'
app.secret_key = 'your_secret_key'

@app.route('/', methods=['GET', 'POST'])
def upload_pdf():
    if request.method == 'POST':
        # Handling file upload
        if 'pdf' not in request.files:
            return redirect(request.url)

        pdf_file = request.files['pdf']
        if pdf_file.filename == '':
            return redirect(request.url)

        if pdf_file and pdf_file.filename.endswith('.pdf'):
            filename = secure_filename(pdf_file.filename)
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            pdf_file.save(file_path)

            # Generate the summary using the Pegasus model or Papermill
            summary_output = generate_legal_summary(file_path)

            # Render the summary result
            return render_template('result.html', summary=summary_output)

    return render_template('index.html')

if __name__ == "__main__":
    if not os.path.exists(app.config['UPLOAD_FOLDER']):
        os.makedirs(app.config['UPLOAD_FOLDER'])
    app.run(debug=True)


Saving FPO7.pdf to FPO7.pdf


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2135: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:3220: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]


Summary of the PDF (with repetitions removed):

The following is a list of some of the most popular stories featured in this week's edition of Sportsmail.com. This week, we look at the latest developments in the world of professional football. In the first part of our series, we take a look at the top 10 teams in the world and see how they compare to each other.


ADDING THE CONTEXT BASED QUESTION AND ANSWER FEATURE USING THE HUGGING FACE TRANSFORMER

In [ ]:
## INSTALLING TRANSFORMERS
!pip install transformers

In [ ]:
!pip install PyPDF2 transformers


In [ ]:
import PyPDF2
from transformers import pipeline
from google.colab import files

# Step 1: Function to extract text from PDF
def extract_text_from_pdf(pdf_file_path):
    with open(pdf_file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text

# Step 2: Set up the question-answering pipeline
def setup_qa_pipeline():
    # Using RoBERTa model fine-tuned on SQuAD2
    qa_pipeline = pipeline(
        "question-answering",
        model="mrm8488/longformer-base-4096-finetuned-squadv2"  # Replace with another model if needed
    )
    return qa_pipeline

# Step 3: Function to answer questions using preloaded context and model
def answer_question(qa_pipeline, context, question):
    # Ask the question
    result = qa_pipeline(question=question, context=context)
    return result['answer']

# Step 4: Main function to handle repetitive Q&A
def main():
    # Upload the PDF file
    uploaded = files.upload()

    # Assuming a single file is uploaded
    for file_name in uploaded.keys():
        pdf_path = file_name
        print(f"PDF file '{pdf_path}' uploaded successfully!")

        # Extract text from the PDF (context)
        context = extract_text_from_pdf(pdf_path)
        print("Text extracted from PDF.")

        # Set up the QA pipeline once
        qa_pipeline = setup_qa_pipeline()
        print("Question-Answering model initialized.")

        # Step 5: Repeated question-answering
        while True:
            question = input("Enter your question (or type 'exit' to stop): ")
            if question.lower() == "exit":
                break

            # Get the answer from the QA pipeline
            answer = answer_question(qa_pipeline, context, question)
            print(f"Answer: {answer}")

# Run the main function
main()


Saving HackOdisha1.pdf to HackOdisha1.pdf
PDF file 'HackOdisha1.pdf' uploaded successfully!
Text extracted from PDF.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/757 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/595M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/longformer-base-4096-finetuned-squadv2 were not used when initializing LongformerForQuestionAnswering: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
- This IS expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Question-Answering model initialized.
Enter your question (or type 'exit' to stop): how to get escape from this situation


Input ids are automatically padded to be a multiple of `config.attention_window`: 512


Answer: no government money
Enter your question (or type 'exit' to stop): e
Answer: 
treated the appellants as equivalent to regular government employees
Enter your question (or type 'exit' to stop): exit
